<a href="https://colab.research.google.com/github/park-hoyeon/park-hoyeon.github.io/blob/master/skt_7_08_OpenCV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  코너(Corner)란 무엇인가
: 코너(Corner)는 영상에서 두 개 이상의 에지가 교차하거나, 픽셀 집합의 방향성이 갑자기 변하는 지점을 의미


In [ ]:
!pip install opencv-python numpy matplotlib --quiet
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
print("OpenCV version:", cv.__version__)


In [ ]:
!wget -q https://raw.githubusercontent.com/opencv/opencv/master/samples/data/left01.jpg -O chessboard.jpg
import os
if not os.path.exists('chessboard.jpg'):
    print("이미지 다운로드 실패. 직접 이미지를 업로드하세요.")
else:
    print("chessboard.jpg 다운로드 완료.")

In [ ]:
img_path = 'chessboard.jpg'
img = cv.imread(img_path)
if img is None:
    raise FileNotFoundError(f"이미지를 불러올 수 없습니다: {img_path}")
img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
plt.figure(figsize=(8,6))
plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
plt.title('Original Chessboard')
plt.axis('off')
plt.show()

In [ ]:
blockSize = 2  # 주변 윈도우 크기
ksize = 3      # Sobel 마스크 크기
k = 0.04       # Harris 파라미터
harris_resp = cv.cornerHarris(img_gray, blockSize, ksize, k)
# 응답값을 보기 좋게 정규화
harris_norm = cv.normalize(harris_resp, None, alpha=0, beta=255, norm_type=cv.NORM_MINMAX)
harris_norm = np.uint8(harris_norm)
thresh = 130
img_harris = img.copy()
for y in range(harris_norm.shape[0]):
    for x in range(harris_norm.shape[1]):
        if harris_norm[y, x] > thresh:
            cv.circle(img_harris, (x, y), 2, (0, 0, 255), -1)
plt.figure(figsize=(8,6))
plt.imshow(cv.cvtColor(img_harris, cv.COLOR_BGR2RGB))
plt.title('Harris Corner Detection')
plt.axis('off')
plt.show()

In [ ]:
max_corners = 100  # 검출할 코너 최대 개수
quality_level = 0.01  # 코너 품질(0~1)
min_distance = 10     # 코너들 간 최소 거리
corners = cv.goodFeaturesToTrack(
    img_gray,
    maxCorners=max_corners,
    qualityLevel=quality_level,
    minDistance=min_distance,
    blockSize=3,
    useHarrisDetector=False
)
img_shi = img.copy()
if corners is not None:
    for pt in corners:
        x, y = pt.ravel()
        cv.circle(img_shi, (int(x), int(y)), 3, (0,255,0), -1)
plt.figure(figsize=(8,6))
plt.imshow(cv.cvtColor(img_shi, cv.COLOR_BGR2RGB))
plt.title('Shi-Tomasi Corner Detection')
plt.axis('off')
plt.show()

In [ ]:
# 첫 번째 이미지를 약간 변형(회전+스케일+이동)하여 두 번째 이미지 생성
rows, cols = img_gray.shape
M = cv.getRotationMatrix2D((cols/2, rows/2), 5, 1.05)  # 중심, 각도=5, 스케일=1.05
M[0,2] += 10  # x방향 이동
M[1,2] += 15  # y방향 이동
img2 = cv.warpAffine(img, M, (cols, rows))
img2_gray = cv.cvtColor(img2, cv.COLOR_BGR2GRAY)
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
plt.title('Frame 1')
plt.axis('off')
plt.subplot(1,2,2)
plt.imshow(cv.cvtColor(img2, cv.COLOR_BGR2RGB))
plt.title('Frame 2 (transformed)')
plt.axis('off')
plt.show()

# Lucas-Kanade 옵티컬 플로우 (Pyramidal)

In [ ]:
feature_params = dict(
    maxCorners = 100,
    qualityLevel = 0.01,
    minDistance = 10,
    blockSize = 3
)
lk_params = dict(
    winSize = (15,15),
    maxLevel = 2,
    criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03)
)
# 첫 번째 프레임의 코너 검출
p0 = cv.goodFeaturesToTrack(img_gray, mask=None, **feature_params)
# calcOpticalFlowPyrLK: (img_gray -> img2_gray)
p1, st, err = cv.calcOpticalFlowPyrLK(img_gray, img2_gray, p0, None, **lk_params)
# st=1인 점만 추출(추적 성공)
good_new = p1[st==1]
good_old = p0[st==1]
img_flow = img2.copy()
for i, (new, old) in enumerate(zip(good_new, good_old)):
    x_new, y_new = new.ravel()
    x_old, y_old = old.ravel()
    cv.circle(img_flow, (int(x_new), int(y_new)), 5, (0,255,0), -1)
    cv.line(img_flow, (int(x_old), int(y_old)), (int(x_new), int(y_new)), (0,0,255), 2)
plt.figure(figsize=(10,6))
plt.imshow(cv.cvtColor(img_flow, cv.COLOR_BGR2RGB))
plt.title('Optical Flow (Frame1 -> Frame2)')
plt.axis('off')
plt.show()

# 종합실습
## 동영상 예시 코


In [ ]:
import cv2 as cv
import numpy as np
import gdown
from google.colab.patches import cv2_imshow
# 비디오 URL 및 로컬 저장 경로 설정
video_url = "https://www.bogotobogo.com/python/OpenCV_Python/images/mean_shift_tracking/slow_traffic_small.mp4"
local_video_path = "slow_traffic_small.mp4"
output_video_path = "optical_flow_output.mp4" # 저장할 비디오 파일 경로
# gdown을 사용하여 비디오 파일 다운로드
gdown.download(video_url, local_video_path, quiet=False)
# 비디오 캡처 객체 생성
cap = cv.VideoCapture(local_video_path)
if not cap.isOpened():
    print("비디오 파일을 열 수 없습니다.")
else:
    # Lucas-Kanade 광학 흐름 파라미터 설정
    lk_params = dict(
        winSize=(15, 15),
        maxLevel=2,
        criteria=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03)
    )
    # 첫 번째 프레임 읽기
    ret, old_frame = cap.read()
    if not ret:
        print("첫 프레임을 읽을 수 없습니다.")
    else:
        old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
        # Shi-Tomasi 코너 검출 파라미터 설정
        feature_params = dict(
            maxCorners=100,
            qualityLevel=0.01,
            minDistance=10,
            blockSize=3
        )
        p0 = cv.goodFeaturesToTrack(old_gray, mask=None, **feature_params)
        # 비디오 저장을 위한 VideoWriter 객체 설정
        fourcc = cv.VideoWriter_fourcc(*'mp4v') # 코덱 설정 (mp4)
        fps = int(cap.get(cv.CAP_PROP_FPS)) # 원본 비디오의 FPS 가져오기
        width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
        out = cv.VideoWriter(output_video_path, fourcc, fps, (width, height))
        print(f"원본 비디오 FPS: {fps}")

        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
            # Lucas-Kanade Optical Flow 계산
            p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
            if p1 is not None:
                good_new = p1[st == 1]
                good_old = p0[st == 1]
                # 시각화용 마스크 생성 및 업데이트
                mask = np.zeros_like(frame) # 각 프레임마다 마스크 초기화
                for i, (new, old) in enumerate(zip(good_new, good_old)):
                    x_new, y_new = new.ravel()
                    x_old, y_old = old.ravel()
                    cv.line(mask, (int(x_new), int(y_new)), (int(x_old), int(y_old)), (0, 255, 0), 2)
                    cv.circle(frame, (int(x_new), int(y_new)), 5, (0, 0, 255), -1)
                img_flow = cv.add(frame, mask)
                out.write(img_flow) # 처리된 프레임을 비디오 파일에 쓰기

            # 다음 프레임을 위해 현재 프레임과 추적된 좋은 특징점 업데이트
            old_gray = frame_gray.copy()
            if p1 is not None: # p1이 None이 아닐 때만 good_new를 업데이트
                 p0 = good_new.reshape(-1, 1, 2)

        # 비디오 처리 완료 후 객체 해제
        cap.release()
        out.release()
        print(f"처리된 비디오가 '{output_video_path}'로 저장되었습니다.")

In [ ]:
import cv2 as cv
import numpy as np
import gdown
from google.colab.patches import cv2_imshow

# 비디오 URL 및 로컬 저장 경로 설정
video_url = "https://www.bogotobogo.com/python/OpenCV_Python/images/mean_shift_tracking/slow_traffic_small.mp4"
local_video_path = "slow_traffic_small.mp4"

# gdown을 사용하여 비디오 파일 다운로드
gdown.download(video_url, local_video_path, quiet=False)

# 비디오 캡처 객체 생성
cap = cv.VideoCapture(local_video_path)

if not cap.isOpened():
    print("비디오 파일을 열 수 없습니다.")
else:
    # Lucas-Kanade Optical Flow 파라미터
    lk_params = dict(
        winSize=(15, 15),
        maxLevel=2,
        criteria=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03)
    )

    # 첫 번째 프레임 읽기
    ret, old_frame = cap.read()
    if not ret:
        print("첫 프레임을 읽을 수 없습니다.")
    else:
        old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)

        # Shi-Tomasi 코너 검출 파라미터
        feature_params = dict(
            maxCorners=100,
            qualityLevel=0.01,
            minDistance=10,
            blockSize=3
        )
        p0 = cv.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

        # 시각화용 마스크 생성
        mask = np.zeros_like(old_frame)

        frame_count = 0  # 너무 많은 프레임 출력 방지용

        while True:
            ret, frame = cap.read()
            if not ret or frame_count > 100:
                break

            frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

            # Lucas-Kanade Optical Flow 계산
            p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

            if p1 is not None:
                good_new = p1[st == 1]
                good_old = p0[st == 1]

                for i, (new, old) in enumerate(zip(good_new, good_old)):
                    x_new, y_new = new.ravel()
                    x_old, y_old = old.ravel()
                    cv.line(mask, (int(x_new), int(y_new)), (int(x_old), int(y_old)), (0, 255, 0), 2)
                    cv.circle(frame, (int(x_new), int(y_new)), 5, (0, 0, 255), -1)

                img_flow = cv.add(frame, mask)
                cv2_imshow(img_flow)

            # 상태 갱신
            old_gray = frame_gray.copy()
            p0 = good_new.reshape(-1, 1, 2)

            frame_count += 1  # 출력 제한을 위한 카운트 증가

        cap.release()


In [ ]:
!pip install opencv-python opencv-contrib-python numpy matplotlib --quiet
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
print("OpenCV version:", cv.__version__)

In [ ]:
!wget -q https://docs.opencv.org/4.x/water_coins.jpg -O coins.png
import os
if not os.path.exists('coins.png'):
    print('coins.png 다운로드 실패. 다른 이미지를 직접 업로드하세요.')
else:
    print('coins.png 다운로드 완료.')

In [ ]:
img_path = "coins.png"
img = cv.imread(img_path)
if img is None:
    raise FileNotFoundError("이미지를 불러올 수 없습니다.")
# BGR -> RGB
img_rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)
# 픽셀을 2차원으로 펼치기
Z = img_rgb.reshape((-1, 3))
Z = np.float32(Z)
# K-means 설정
K = 3
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 10, 1.0)
attempts = 10  # 다른 초기값 시도 횟수
flags = cv.KMEANS_PP_CENTERS  # K-means++ 초기화
compactness, labels, centers = cv.kmeans(Z, K, None, criteria, attempts, flags)

# centers: (K, 3) 형태의 클러스터 중심
centers = np.uint8(centers)
segmented = centers[labels.flatten()]
segmented_img = segmented.reshape(img_rgb.shape)
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.imshow(img_rgb)
plt.title('Original')
plt.axis('off')
plt.subplot(1,2,2)
plt.imshow(segmented_img)
plt.title('K-means K=3')
plt.axis('off')
plt.show()


In [ ]:
from cv2.ximgproc import segmentation as xseg
# 1. 세그먼테이션 객체 생성
sigma = 0.5
k = 300
min_size = 100
graph_seg = xseg.createGraphSegmentation(sigma=sigma, k=k, min_size=min_size)
# 2. 이미지 처리
labels = graph_seg.processImage(img)  # BGR 이미지 입력
num_segments = labels.max() + 1
print("Number of segments:", num_segments)
# 3. 세그먼트 별 랜덤 컬러로 시각화
seg_img = np.zeros_like(img)
colors = [np.random.randint(0,255,3, dtype=np.uint8) for _ in range(num_segments)]
for seg_val in range(num_segments):
    seg_img[labels == seg_val] = colors[seg_val]
seg_img_rgb = cv.cvtColor(seg_img, cv.COLOR_BGR2RGB)
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
plt.title('Original')
plt.axis('off')
plt.subplot(1,2,2)
plt.imshow(seg_img_rgb)
plt.title('Graph-based Segmentation')
plt.axis('off')
plt.show()

In [ ]:
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
# 1. 이진화 (Otsu + Inverse)
ret, thresh = cv.threshold(gray, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)
# 2. 노이즈 제거 -> 침식/팽창(열림)
kernel = np.ones((3,3), np.uint8)
opening = cv.morphologyEx(thresh, cv.MORPH_OPEN, kernel, iterations=2)
# 3. 확실한 배경
sure_bg = cv.dilate(opening, kernel, iterations=3)
# 4. 확실한 전경 -> Distance Transform 후, 임계값
dist_transform = cv.distanceTransform(opening, cv.DIST_L2, 5)
ret, sure_fg = cv.threshold(dist_transform, 0.7*dist_transform.max(), 255, 0)
sure_fg = np.uint8(sure_fg)
# 5. 불확실 영역
unknown = cv.subtract(sure_bg, sure_fg)
# 6. 전경을 Connected Components로 라벨
ret, markers = cv.connectedComponents(sure_fg)
markers = markers + 1  # 배경을 1, 나머지는 2,3,...
markers[unknown==255] = 0
# 7. Watershed
watershed_img = img.copy()
markers = cv.watershed(watershed_img, markers)
watershed_img[markers == -1] = [0,0,255]  # 경계 표시
# 시각화
fig, axs = plt.subplots(1,3, figsize=(15,5))
axs[0].imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
axs[0].set_title('Original')
axs[0].axis('off')
axs[1].imshow(thresh, cmap='gray')
axs[1].set_title('Threshold')
axs[1].axis('off')
axs[2].imshow(cv.cvtColor(watershed_img, cv.COLOR_BGR2RGB))
axs[2].set_title('Watershed Result')
axs[2].axis('off')
plt.show()

# OpenCV와 KNN을 활용한 이미지 분류

In [ ]:
!pip install opencv-python-headless


In [ ]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
print("OpenCV version:", cv2.__version__)

In [ ]:
knn = cv2.ml.KNearest_create()

# knn 모델 훈련 (trainData와 trainLabels는 이전에 준비되어야 합니다)
# 아래 Agt5TXaZzHcg 셀에서 train과 train_labels를 사용합니다.
# Agt5TXaZzHcg 셀이 먼저 실행되어 train과 train_labels가 정의되어 있어야 합니다.
if 'train' not in globals() or 'train_labels' not in globals():
    print("오류: 'train' 또는 'train_labels' 변수가 정의되지 않았습니다.")
    print("이전 셀(Agt5TXaZzHcg)을 먼저 실행하여 훈련 데이터를 로드하고 준비해주세요.")
else:
    trainData = train # Agt5TXaZzHcg 셀에서 정의된 train 사용
    trainLabels = train_labels # Agt5TXaZzHcg 셀에서 정의된 train_labels 사용

    knn.train(trainData, cv2.ml.ROW_SAMPLE, trainLabels)

    # 분류할 샘플 데이터 준비
    # 여기서는 digits.png 이미지의 일부를 테스트 데이터로 사용합니다.
    # 실제 사용 시에는 분류하고자 하는 새로운 이미지 데이터로 대체해야 합니다.

    # img_digits 변수가 정의되었는지 확인
    if 'img_digits' not in globals() or img_digits is None:
         print("오류: 'img_digits' 변수가 정의되지 않았거나 이미지를 불러오지 못했습니다.")
         print("이전 셀(Agt5TXaZzHcg)을 먼저 실행하여 digits.png 이미지를 로드해주세요.")
    else:
        # digits.png 이미지에서 일부 셀을 테스트 데이터로 사용 (예: 각 숫자 500개 중 마지막 100개)
        test_cells = [np.hsplit(row, 100)[40:50] for row in np.vsplit(img_digits, 50)] # 각 숫자당 10개씩 사용
        test_x = np.array(test_cells) # shape (50, 10, 20, 20)
        sampleData = test_x.reshape(-1, 400).astype(np.float32) # (50*10, 400) = (500, 400)

        # 테스트 데이터에 대한 실제 레이블 (검증용)
        test_labels = np.repeat(np.arange(10), 10)[:, np.newaxis] # (500, 1)

        k = 5 # 가장 가까운 k개의 이웃 사용

        # k개의 가장 가까운 이웃 찾기
        # ret: 결과 레이블, result: 예측된 레이블, neighbours: 가장 가까운 이웃의 레이블, dist: 이웃까지의 거리
        ret, result, neighbours, dist = knn.findNearest(sampleData, k)

        # 예측 결과 평가 (테스트 레이블이 있는 경우)
        if test_labels is not None:
            # 예측 결과와 실제 레이블 비교
            matches = result == test_labels
            correct = np.count_nonzero(matches)
            accuracy = correct * 100.0 / result.size
            print(f"Accuracy: {accuracy:.2f}%")

        # 결과 출력 (샘플 데이터가 비어있지 않으므로 이제 출력 가능)
        # print("Result:", result) # 예측된 레이블
        # print("Neighbours:", neighbours) # 가장 가까운 이웃의 레이블
        # print("Distance:", dist) # 이웃까지의 거리

In [ ]:
# 3.1 digits.png 로드 (그레이스케일)
# wget으로 파일 다운로드
url = "https://raw.githubusercontent.com/npinto/opencv/master/samples/python2/data/digits.png"
output_file = "digits.png"
os.system(f"wget -O {output_file} {url}")
# OpenCV로 이미지 읽기
img_digits = cv2.imread(output_file, cv2.IMREAD_GRAYSCALE)
if img_digits is None:
    print("digits.png를 불러올 수 없습니다.")
else:
    print(f"Original Image Shape: {img_digits.shape}")
    plt.imshow(img_digits, cmap='gray')
    plt.title("Original Digits Image")
    plt.axis('off')
    plt.show()
    # 3.2 이미지 분할
    # 세로 50줄, 가로 100줄로
    cells = [np.hsplit(row, 100) for row in np.vsplit(img_digits, 50)]
    x = np.array(cells)  # shape (50, 100, 20, 20)
    # 각 셀(20x20)을 1x400 벡터로 변환
    train = x.reshape(-1, 400).astype(np.float32)  # (50*100, 400)
    # 레이블 생성 (0~9 각각 500번 = 5000)
    k = np.arange(10)
    train_labels = np.repeat(k, 500)[:, np.newaxis]
      # 데이터 저장
    np.savez("trained_opencv.npz", train=train, train_labels=train_labels)
    print("Training data saved.")

# 4. 이미지 전처리 함수 (OpenCV 활용)


In [ ]:
def preprocess_image_colab(image_path):
    """
    테스트 이미지(예: 숫자 1장) 전처리:
    1) GRAYSCALE 로드
    2) 이진화
    3) 외곽선 표시 (시각화용)
    4) 20x20 크기로 리사이즈
    5) 1x400 형태로 벡터화 (float32)
    """
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"Image not found at {image_path}")
    # 이진화
    _, binary = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY_INV)
    # 외곽선 검출
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    img_with_contours = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    cv2.drawContours(img_with_contours, contours, -1, (0,255,0), 2)
    # 크기 조정(20x20)
    resized = cv2.resize(binary, (20, 20))
    # 시각화 (Colab)
    print("[Original Grayscale]")
    cv2_imshow(img)
    print("[Binary Inverted]")
    cv2_imshow(binary)
    print("[Contours Drawn]")
    cv2_imshow(img_with_contours)
    print("[Resized 20x20]")
    cv2_imshow(resized)
    # reshape(1, 400), float32
    vector_400 = resized.reshape(-1, 400).astype(np.float32)
    return vector_400


#  5. KNN 알고리즘 및 테스트


In [ ]:
def knn_train_and_test_colab(train_data, train_labels, test_image_path, k=3):
    # 1) KNN 모델 생성
    knn = cv2.ml.KNearest_create()
    # 2) 모델 학습
    # train_data.shape: (5000, 400), train_labels.shape: (5000,1)
    knn.train(train_data, cv2.ml.ROW_SAMPLE, train_labels)
    # 3) 테스트 이미지 전처리
    test_vector = preprocess_image_colab(test_image_path)  # (1, 400)
    # 4) 예측
    ret, result, neighbours, dist = knn.findNearest(test_vector, k)
    predicted_label = int(result[0][0])
    print(f"Predicted Label: {predicted_label}")
    print(f"Neighbours: {neighbours}")
    print(f"Distances: {dist}")
    return predicted_label, neighbours, dist


#  5.1 학습 데이터 로드


In [ ]:
# 학습 데이터 로드
train_data = None
train_labels = None
try:
    with np.load('trained_opencv.npz') as data:
        train_data = data['train']
        train_labels = data['train_labels']
    print(f"train_data.shape = {train_data.shape}, train_labels.shape = {train_labels.shape}")
except FileNotFoundError:
    print("trained_opencv.npz 파일이 존재하지 않습니다. 앞 셀을 실행해주세요.")

    # 테스트 이미지 (예: test_digit_3.png) - digits.png의 한 셀을 잘라 저장했다고 가정
test_image_path = 'test_digit_3.png'  # Colab 환경에 업로드된 테스트 이미지
pred_label, neighbours, distances = None, None, None
if train_data is not None and train_labels is not None:
    pred_label, neighbours, distances = knn_train_and_test_colab(
        train_data, train_labels, test_image_path, k=5
    )


In [ ]:
# 테스트 이미지 (예: test_digit_3.png) - digits.png의 한 셀을 잘라 저장했다고 가정
test_image_path = '귀여움.jpg'  # Colab 환경에 업로드된 테스트 이미지
pred_label, neighbours, distances = None, None, None
if train_data is not None and train_labels is not None:
    pred_label, neighbours, distances = knn_train_and_test_colab(
        train_data, train_labels, test_image_path, k=5
    )

# 6. 결과 시각화 (예측 결과 표시)

In [ ]:
def visualize_result_colab(image_path, predicted_label):
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Image not found at {image_path}")
    output = img.copy()
    text = f"Predicted: {predicted_label}"
    cv2.putText(
        output, text, (10,50),
        cv2.FONT_HERSHEY_SIMPLEX, 1,
        (0,255,0), 2
    )
    print("[Test Image with Prediction]")
    cv2_imshow(output)


In [ ]:
# 결과 시각화
if pred_label is not None:
    visualize_result_colab(test_image_path, pred_label)

#  7.1 과제 결과 예시 코드: K 값 루프


In [ ]:
def test_knn_with_different_k(train_data, train_labels, test_image_data, k_values=[1,3,5,7,9]):
    """
    여러 테스트 이미지 데이터에 대해 KNN 분류를 수행하고 결과를 반환합니다.
    test_image_data는 (이미지 벡터, 실제 라벨) 튜플의 리스트입니다.
    """
    results = {}
    for k_ in k_values:
        print(f"\n=== K={k_} ===")
        knn = cv2.ml.KNearest_create()
        knn.train(train_data, cv2.ml.ROW_SAMPLE, train_labels)
        pred_labels = []
        correct_count = 0
        total_count = len(test_image_data)

        for test_vec, true_label in test_image_data:
            # preprocess_image_colab 함수는 파일 경로를 받으므로,
            # 여기서는 이미 전처리된 벡터를 사용하도록 로직 변경
            ret, result, neigh, dist = knn.findNearest(test_vec, k_)
            plabel = int(result[0][0])
            pred_labels.append(plabel)
            # print(f"True Label={true_label}, Predicted={plabel}") # 개별 결과 출력 (선택 사항)

            if plabel == true_label:
                correct_count += 1

        accuracy = (correct_count / total_count) * 100 if total_count > 0 else 0
        print(f"Accuracy for K={k_}: {accuracy:.2f}% ({correct_count}/{total_count})")
        results[k_] = {'predicted_labels': pred_labels, 'accuracy': accuracy}

    return results

## 예시 사용
# digits.png 이미지에서 테스트 데이터 추출
# img_digits 변수가 Agt5TXaZzHcg 셀에서 로드되었다고 가정
if 'img_digits' in globals() and img_digits is not None:
    # 각 숫자(0-9) 당 마지막 10개 샘플을 테스트 데이터로 사용
    # cells = [np.hsplit(row, 100) for row in np.vsplit(img_digits, 50)]
    # test_cells_raw = [cells[i][-10:] for i in range(50)] # 각 row의 마지막 10개
    # test_cells_flat = [item for sublist in test_cells_raw for item in sublist] # flatten list
    # test_x_raw = np.array(test_cells_flat) # shape (500, 20, 20)

    # 좀 더 간단하게, train 데이터에서 일부를 test 데이터로 분리 (예: 마지막 1000개)
    # 전체 5000개 데이터 중 마지막 1000개 (각 숫자당 100개씩)를 테스트 데이터로 사용
    train_data_subset = train[-1000:]
    train_labels_subset = train_labels[-1000:]

    # 테스트 이미지 데이터 형식: (이미지 벡터, 실제 라벨) 튜플의 리스트
    test_imgs_data = [(train_data_subset[i].reshape(1, -1), int(train_labels_subset[i][0])) for i in range(len(train_data_subset))]


    if train_data is not None and train_labels is not None:
        # k=1,3,5 등 루프
        _results = test_knn_with_different_k(train_data, train_labels, test_imgs_data, k_values=[1,3,5])
        print("\nKNN test done!")
else:
    print("오류: 'img_digits' 변수가 정의되지 않았거나 이미지를 불러오지 못했습니다.")
    print("이전 셀(Agt5TXaZzHcg)을 먼저 실행하여 digits.png 이미지를 로드해주세요.")